In [41]:
# Import necessary libraries and packages
import json
import heapq

In [42]:
# Load json files
def load_json_file(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

Cost = load_json_file("Cost.json")
G = load_json_file("G.json")
Coord = load_json_file("Coord.json")
Dist = load_json_file("Dist.json")

## Understanding the data 

In [43]:
# Check type of the loaded data
print(type(Cost))
print(type(G))
print(type(Coord))
print(type(Dist))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [44]:
# Check the length of the dictionaries
print(f"Length of Cost dictionary: {len(Cost)}") if isinstance(Cost, (list, dict)) else "Not a list or dict"
print(f"Length of G dictionary: {len(G)}") if isinstance(G, (list, dict)) else "Not a list or dict"
print(f"Length of Coord dictionary: {len(Coord)}") if isinstance(Coord, (list, dict)) else "Not a list or dict"
print(f"Length of Dist dictionary: {len(Dist)}") if isinstance(Dist, (list, dict)) else "Not a list or dict"

Length of Cost dictionary: 730100
Length of G dictionary: 264346
Length of Coord dictionary: 264346
Length of Dist dictionary: 730100


### Graph dictionary G
- The graph is given as an adjacency list where the neighbor list of node ‘v’ can be accessed with G[‘v’].


In [45]:
# Structure of G dictionary
def print_structure(data, iterations=5):
    i = 0
    if isinstance(data, dict):
        for key in data:
            print("Key: ", str(key))
            print_structure(data[key])
            i += 1
            if i >= iterations:
                return
    else:
        print(str(type(data)) + ' ' + str(data))
    
print_structure(G)

Key:  1
<class 'list'> ['1363', '12', '2']
Key:  2
<class 'list'> ['13', '1', '48']
Key:  3
<class 'list'> ['4', '3874']
Key:  4
<class 'list'> ['3926', '3', '3937']
Key:  5
<class 'list'> ['6', '1204', '1214', '1219']


In [46]:
# Example usage of G dictionary
print("Neighbouring nodes of node 1: ", G["1"])

Neighbouring nodes of node 1:  ['1363', '12', '2']


### Node coordination dictionary Coord
- The coordination of a node ‘v’ is a pair (X, Y) which can be accessed with Coord[‘v’]


In [47]:
print_structure(Coord)

Key:  1
<class 'list'> [-73530767, 41085396]
Key:  2
<class 'list'> [-73530538, 41086098]
Key:  3
<class 'list'> [-73519366, 41048796]
Key:  4
<class 'list'> [-73519377, 41048654]
Key:  5
<class 'list'> [-73524567, 41093796]


In [48]:
# Example usage of Coord dictionary
print("Coordinates of node 1: ", Coord["1"])
print("Coordinates of node 2: ", Coord["2"])

Coordinates of node 1:  [-73530767, 41085396]
Coordinates of node 2:  [-73530538, 41086098]


### Edge distance dictionary Dist
The distance between a pair of node (v, w) can be accessed with Dist[‘v,w’].

In [49]:
print_structure(Dist)

Key:  1,2
<class 'int'> 803
Key:  2,1
<class 'int'> 803
Key:  3,4
<class 'int'> 158
Key:  4,3
<class 'int'> 158
Key:  5,6
<class 'float'> 923.7819006670352


In [50]:
# Example usage of Dist dictionary
print("Distance between nodes 1 and 2: ", Dist["1,2"])

Distance between nodes 1 and 2:  803


### Edge cost dictionary Cost
The energy cost between a pair of node (v, w) can be accessed with Cost[‘v,w’.]

In [51]:
print_structure(Cost)

Key:  1,2
<class 'int'> 2008
Key:  2,1
<class 'int'> 2008
Key:  3,4
<class 'int'> 395
Key:  4,3
<class 'int'> 395
Key:  5,6
<class 'int'> 1935


In [52]:
# Example usage of Cost dictionary
print("Cost of travelling between node 1 and 2: ", Cost["1,2"])

Cost of travelling between node 1 and 2:  2008


### Combining the graphs together
- except coordinates

In [53]:
combined_graph = {}

for node, neighbors in G.items():
    combined_graph[node] = []
    for neighbor in neighbors:
        edge_key = f"{node},{neighbor}"
        edge_data = {
            "node": neighbor,
            "distance": Dist.get(edge_key, float('inf')), # Using inf as a default if no distance is provided
            "cost": Cost.get(edge_key, float('inf')) # Using inf as a default if no cost is provided
        }
        combined_graph[node].append(edge_data)

print_structure(combined_graph)


Key:  1
<class 'list'> [{'node': '1363', 'distance': 2428, 'cost': 6070}, {'node': '12', 'distance': 1004.7188661511238, 'cost': 2105}, {'node': '2', 'distance': 803, 'cost': 2008}]
Key:  2
<class 'list'> [{'node': '13', 'distance': 704.9198536003934, 'cost': 1478}, {'node': '1', 'distance': 803, 'cost': 2008}, {'node': '48', 'distance': 617, 'cost': 1541}]
Key:  3
<class 'list'> [{'node': '4', 'distance': 158, 'cost': 395}, {'node': '3874', 'distance': 1667, 'cost': 4167}]
Key:  4
<class 'list'> [{'node': '3926', 'distance': 294.7626163542453, 'cost': 627}, {'node': '3', 'distance': 158, 'cost': 395}, {'node': '3937', 'distance': 725.041378129552, 'cost': 1541}]
Key:  5
<class 'list'> [{'node': '6', 'distance': 923.7819006670352, 'cost': 1935}, {'node': '1204', 'distance': 2020.3960007879643, 'cost': 4801}, {'node': '1214', 'distance': 1603.1219541881396, 'cost': 3361}, {'node': '1219', 'distance': 3382, 'cost': 8456}]


# Task 1

You will need to solve a relaxed version of the NYC instance where we do not have the energy
constraint. You can use any algorithm we discussed in the lectures. Note that this is equivalent to solving the shortest path problem. The solution quality of your algorithm will affect the grade.

### Solution: Dijkstra's Algorithm

- Dijkstra's algorithm finds the shortest path from one node to every other node in the graph. (or in our case from one node (S) to a target node(T)) 
- It starts with a selected initial node & it attempts to find the shortest path to all the other nodes
- Quick youtube explanation: https://www.youtube.com/watch?v=_lHSawdgXpI&ab_channel=MichaelSambol

1. Initialisation:
    - Set the initial node's distance to zero and all other nodes' distances to infinity.
    - Maintain a priority queue/set of all the nodes, with nodes prioritized by their current known distance from the start node.

2. Iterative exploration:
    - Always choose the next node, u, with the shortest known distance from the priority queue (i.e., the node at the front of the queue). This is the node we'll explore next.
    - For each of u's neighbors, v, check if the path to v via u is shorter than any previously known path to v. If it is, update v's distance and potentially its position in the priority queue.
    - Remove u from the priority queue.
    - Repeat the process until the priority queue is empty or you find the target node (dequeue the target node from queue).
    
** The heap (or priority queue) is crucial because it always provides the node with the current shortest distance without having to scan all nodes. (This is why the algorithm is efficient)

In [57]:
def dijkstra(graph, start, target):
    # Initialize distances with infinite values and predecessors to None
    distances = {node: float('infinity') for node in graph} # find shortest distance
    predecessors = {node: None for node in graph} # keep track of path
    distances[start] = 0 # initialise distance of source node to 0 (since shortest path to source node is 0)
    priority_queue = [(0, start)] # For efficient look up O(1) of next node to explore

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop condition for single source-target shortest path
        if current_node == target:
            break

        # Check if current distance is smaller in our set
        if current_distance > distances[current_node]:
            continue

        for neighbor in graph[current_node]:
            distance = current_distance + neighbor["distance"]
            if distance < distances[neighbor["node"]]:
                distances[neighbor["node"]] = distance
                predecessors[neighbor["node"]] = current_node
                heapq.heappush(priority_queue, (distance, neighbor["node"]))

    # Reconstruct the shortest path from the predecessor dictionary
    path = []
    while target:
        path.append(target)
        target = predecessors[target]
    path.reverse()  # Reverse to get path from start to target

    # Calculate the total energy cost
    total_energy_cost = 0
    for i in range(len(path) - 1):
        for neighbor in graph[path[i]]:
            if neighbor["node"] == path[i + 1]:
                total_energy_cost += neighbor["cost"]
                break

    return distances[path[-1]], path, total_energy_cost

start_node = "1"
target_node = "50"
shortest_distance, shortest_path, energy_cost = dijkstra(combined_graph, start_node, target_node)
print("Shortest path:", "->".join(shortest_path))
print(f"Shortest distance: {shortest_distance}")
print(f"Total Energy Cost: {energy_cost}")


Shortest path: 1->1363->1358->1357->1356->1276->1273->1277->1269->1267->1268->1284->1283->1282->1255->1253->1260->1259->1249->1246->963->964->962->1002->952->1000->998->994->995->996->987->988->979->980->969->977->989->990->991->2369->2366->2340->2338->2339->2333->2334->2329->2029->2027->2019->2022->2000->1996->1997->1993->1992->1989->1984->2001->1900->1875->1874->1965->1963->1964->1923->1944->1945->1938->1937->1939->1935->1931->1934->1673->1675->1674->1837->1671->1828->1825->1817->1815->1634->1814->1813->1632->1631->1742->1741->1740->1739->1591->1689->1585->1584->1688->1579->1679->1677->104->5680->5418->5431->5425->5424->5422->5413->5412->5411->66->5392->5391->5388->5291->5278->5289->5290->5283->5284->5280->50
Shortest distance: 148648.63722140007
Total Energy Cost: 294853


# Task 2

You will need to implement an uninformed search algorithm (e.g., the DFS, BFS, UCS) to solve
the NYC instance. For tasks 2 and 3, the energy budget is set to be 287932.

### Solution: Uniform Cost Search Algorithm

- UCS explores the graph based on the cost of the path from the start node to the current node.
- We use UCS as our search algo, but with an added check for the energy constraint. 
- We maintain two costs: The distance (to prioritize path exploration) & the energy cost (to check against the budget constraint).

Why UCS?
- Prunes path that exceed budget early - saving time as compared to BFS & DFS
- Explores path in order of increasing distance (similar to dijkstra) to ensure efficiency in finding shortest path

Steps:

1. Initialise source node with distance = 0 and energy_cost = 0
2. Use a priority queue (heap) to explore nodes efficiently, prioritising distance (since we are still trying to find the shortest path)
3. Expand nodes, adding the distance and energy_cost for each child
4. If the total_energy_cost > energy_budget , we do not enqueue the child node
5. If target node is dequeued (shortest distance in the heap), and energy constraint is satisfied, we return the path found


In [58]:
def ucs_with_energy_constraint(graph, start, target, energy_budget):
    visited = set()
    # Each item in the priority queue is (distance, energy_cost, current_node, path_so_far)
    priority_queue = [(0, 0, start, [])]

    while priority_queue:
        current_distance, current_energy_cost, current_node, path = heapq.heappop(priority_queue)

        if current_node in visited:
            continue

        new_path = path + [current_node]

        # If we reach the target node and satisfy the energy constraint, return the path, distance, and energy cost
        if current_node == target:
            return new_path, current_distance, current_energy_cost

        visited.add(current_node)

        for neighbor in graph[current_node]:
            if neighbor["node"] not in visited:
                # Sum the distance and energy cost to get to this neighbor
                new_distance = current_distance + neighbor["distance"]
                new_energy_cost = current_energy_cost + neighbor["cost"]

                # Only enqueue if the new energy cost is within the budget
                if new_energy_cost <= energy_budget:
                    heapq.heappush(priority_queue, (new_distance, new_energy_cost, neighbor["node"], new_path))

    # Return None if no path found within the energy constraint
    return None, None, None

energy_budget = 287932
start_node = "1"
target_node = "50"
shortest_path, shortest_distance, total_energy_cost = ucs_with_energy_constraint(combined_graph, start_node, target_node, energy_budget)
if shortest_path:
    print("Shortest path within energy constraint:", "->".join(shortest_path))
    print(f"Total Distance Traveled: {shortest_distance}")
    print(f"Total Energy Cost: {total_energy_cost}")
else:
    print("No path found within the given energy constraint.")


Shortest path within energy constraint: 1->1363->1358->1357->1356->1276->1273->1277->1269->1267->1268->1284->1283->1282->1255->1253->1260->1259->1249->1246->963->964->962->1002->952->1000->998->994->995->996->987->986->979->980->969->977->989->990->991->2369->2366->2340->2338->2339->2333->2334->2329->2029->2027->2019->2022->2000->1996->1997->1993->1992->1989->1984->2001->1900->1875->1874->1965->1963->1964->1923->1944->1945->1938->1937->1939->1935->1931->1934->1673->1675->1674->1837->1671->1828->1825->1817->1815->1634->1814->1813->1632->1631->1742->1741->1740->1739->1591->1689->1585->1584->1688->1579->1679->1677->104->5680->5418->5431->5425->5429->5426->5428->5434->5435->5433->5436->5398->5404->5402->5396->5395->5292->5282->5283->5284->5280->50
Total Distance Traveled: 150784.60722193593
Total Energy Cost: 287931
